In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

def relu(x):
    """
    ReLU activation function.

    Parameters:
    - x: Input values.

    Returns:
    - (max(0, x)).
    """
    return np.maximum(0, x)

def relu_deriv(x):
    """
    Computes the derivative of the ReLU function.

    Parameters:
    - x: Input values.

    Returns:
    - Derivative of ReLU
    """
    return (x > 0).astype(float)

def sigmoid(x):
    """
     sigmoid activation function.

    Parameters:
    - x: Input values.

    Returns:
    - Activated values using the sigmoid function.
    """
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    """
    Computes the derivative of the sigmoid function.

    Parameters:
    - x: Input values.

    Returns:
    - Derivative of the sigmoid function.
    """
    s = sigmoid(x)
    return s * (1 - s)

# ---------- Noise Addition Function ----------

def add_noise(X, factor=0.2):
    """
    Adds Gaussian noise to the input data.

    Parameters:
    - X: Original input data.
    - factor: Noise level to apply.

    Returns:
    - Noisy version of the input data.range [0, 1].
    """
    return np.clip(X + factor * np.random.randn(*X.shape), 0., 1.)

### **Autoencoder**

In [6]:
class Autoencoder:
    """
    A basic feedforward autoencoder for denoising input data.
    """

    def __init__(self, layers, lr=0.01, epochs=20, batch_size=64):
        """
        Initializes the autoencoder.

        Parameters:
        - layers: A list representing the number of neurons in each layer.
        - lr: Learning rate for training.
        - epochs: Number of training iterations.
        - batch_size: Number of samples in each batch.
        """
        self.lr = lr
        self.layers = layers
        self.weights = [np.random.randn(layers[i], layers[i+1]) * 0.01 for i in range(len(layers)-1)]
        self.biases  = [np.zeros((1, layers[i+1])) for i in range(len(layers)-1)]
        self.epochs = epochs
        self.batch_size = batch_size

    def _forward(self, X):
        """
        Performs the forward pass through the network.

        Parameters:
        - X: Input data.

        Returns:
        - Z: List of linear outputs before activation.
        - A: List of activations after each layer.
        """
        Z, A = [], [X]
        for i in range(len(self.weights)-1):
            Z.append(A[-1] @ self.weights[i] + self.biases[i])
            A.append(relu(Z[-1]))
        Z.append(A[-1] @ self.weights[-1] + self.biases[-1])
        A.append(sigmoid(Z[-1]))
        return Z, A

    def _backward(self, Z, A, Y):
        """
        Performs the backward pass and computes gradients.

        Parameters:
        - Z: Outputs before activation from forward pass.
        - A: Activations from forward pass.
        - Y: Ground truth (clean) data.

        Returns:
        - grads_w: Gradients of the weights.
        - grads_b: Gradients of the biases.
        """
        grads_w, grads_b = [], []
        delta = (A[-1] - Y) * sigmoid_deriv(Z[-1])
        for i in reversed(range(len(self.weights))):
            grads_w.insert(0, A[i].T @ delta)
            grads_b.insert(0, np.sum(delta, axis=0, keepdims=True))
            if i > 0:
                delta = (delta @ self.weights[i].T) * relu_deriv(Z[i-1])
        for j in range(len(self.weights)):
            self.weights[j] -= self.lr * grads_w[j]
            self.biases[j] -= self.lr * grads_b[j]


    def fit(self, X_clean, X_noisy):
        """
        Trains the autoencoder using mini-batch gradient descent.

        Parameters:
        - X_clean: Clean input data (ground truth).
        - X_noisy: Noisy version of the input data.
        """
        n = X_clean.shape[0]
        for epoch in range(self.epochs):
            idx = np.random.permutation(n)
            X_clean, X_noisy = X_clean[idx], X_noisy[idx]
            for i in range(0, n, self.batch_size):
                xb, yb = X_noisy[i:i+self.batch_size], X_clean[i:i+self.batch_size]
                Z, A = self._forward(xb)
                self._backward(Z, A, yb)


    def predict(self, X):
        """
        Uses the trained autoencoder to reconstruct input data.

        Parameters:
        - X: Input data to reconstruct.

        Returns:
        - The reconstructed version of the input.
        """
        _, A = self._forward(X)
        return A[-1]





**Main**

In [7]:
if __name__ == '__main__':
    # Load the MNIST datasets
    data_train = pd.read_csv("MNIST-train.csv").to_numpy()
    data_test = pd.read_csv("MNIST-test.csv").to_numpy()

    # Split features and labels
    X_train, y_train = data_train[:, :-1], data_train[:, -1]
    X_test, y_test = data_test[:, :-1], data_test[:, -1]

    # Normalize pixel values
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    # Add noise
    X_train_noisy = add_noise(X_train, 0.2)
    X_test_noisy  = add_noise(X_test, 0.2)

    # Train Autoencoder
    autoencoder = Autoencoder(layers=[784, 128, 64, 32, 64, 128, 784], lr=0.01)
    autoencoder.fit(X_train, X_train_noisy)

    # Evaluate on test set
    reconstructed = autoencoder.predict(X_test_noisy)
    print(f"\nTest MSE: {mean_squared_error(X_test, reconstructed):.6f}")


Test MSE: 0.067575
